In [12]:
using JSON
using Statistics
using CSV
using Tables
using DataFrames
using LightGraphs
include("utils.jl")
include("media.jl")

deffuant_bias_media (generic function with 1 method)

In [3]:
function keys_to_int(dict)
    newdict = Dict([parse(Int, string(key)) => val for (key, val) in pairs(dict)])    
    return newdict
end

function splitstring(s)
#     println("string=$s")
    v = split(s, r"\(|\)| ")
    nums = []
    for i = 1:length(v)
        try
            n = parse(Float16, v[i])
#             println("n=$n")
            append!(nums, n)
        catch
            continue
        end
    end
#     println("nums=$nums")
    return Tuple([Float16(nums[1]), Float16(nums[2])])
end

function read_json(filename)
    inDict = JSON.parsefile(filename)
    newdict = keys_to_int(inDict)
    newnewdict = Dict()
    for nr in keys(newdict)
        newnewdict[nr] = Dict()
        dicttoadd = keys_to_tuple(newdict[nr])
        newnewdict[nr] = dicttoadd
    end
    return newnewdict
end

function read_json_cluster(filename)
    inDict = JSON.parsefile(filename)
    newdict = keys_to_int(inDict)
    newnewdict = Dict()
    for nr in keys(newdict)
        newnewdict[nr] = Dict()
        dicttoadd = keys_to_tuple(newdict[nr])
        newnewdict[nr] = dicttoadd
    end
    return newnewdict
end

read_json_cluster (generic function with 1 method)

In [4]:
function keys_to_tuple(dict)
    newdict = Dict([splitstring(key) => val for (key, val) in pairs(dict)])    
    return newdict
end

keys_to_tuple (generic function with 1 method)

In [ ]:
for (root, dirs, files) in walkdir("aggregate/")
    for file in files
        if startswith(file, "final_clusters") 
            final_clusters = read_json_cluster("aggregate/$file")  
            for k in keys(final_clusters)
                println(k)
            end
            json_string = JSON.json(final_clusters)
            open("aggregate/$file ", "w") do f
#                 write(f, json_string)
            end
        end
    end
end


In [5]:
inDict = JSON.parsefile("aggregate/final_clusters media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.25 gm0.25 mi1000000.json")
newdict = keys_to_int(inDict)
newnewdict = Dict()
for nr in keys(newdict)
    newnewdict[nr] = Dict()
    dicttoadd = keys_to_tuple(newdict[nr])
    newnewdict[nr] = dicttoadd
end
newnewdict

LoadError: Unexpected end of input
Line: 0
Around: ......
           ^


In [6]:
final_clusters = newnewdict

LoadError: UndefVarError: newnewdict not defined

In [19]:
filename = "aggregate/final_clusters media mo[0.05, 0.5, 0.95] p0.1 e0.2 g0.5 gm0.5 mi1000000.json"

"aggregate/final_clusters media mo[0.05, 0.5, 0.95] p0.1 e0.2 g0.5 gm0.5 mi1000000.json"

In [29]:
function read_json_cluster(filename)
    #leggo il json con i cluster
    inDict = JSON.parsefile(filename, use_mmap=false)
    println(inDict)
    #trasformo le chiavi (nr) in interi
    newdict = keys_to_int(inDict)
    #creo un dizionario vuoto
    newnewdict = Dict()
    for nr in keys(newdict)
        #per ogni chiave creo un dizionario vuoto dentro l'altro dizionario vuoto
        # newnewdict[nr] = Dict()
        #aggiungo il dizionario corrispondente alla nr con le chiavi trasformate da stringhe a tuple
        dicttoadd = keys_to_tuple(newdict[nr])
        newnewdict[nr] = dicttoadd
    end
    return newnewdict
end

nothing


read_json_cluster (generic function with 1 method)

In [31]:
function return_dictionaries(f, name; nruns=100)

    function create_dictionaries()
        if isfile("aggregate/final_clusters $name.json")
            final_clusters = read_json_cluster("aggregate/final_clusters $name.json")
        else
            println("aggregate files not present")
            final_clusters = Dict()
        end

        return final_clusters
    end

    final_clusters = create_dictionaries()
    
    fc = final_clusters

    for nr in 1:nruns
        if nr in keys(fc)
            continue
        else
            return
        end
        o = [x for x in r[size(r)[1]]]
        clusters = population_clusters([x for x in r[end]])
        merge!(fc, Dict(nr=>clusters))
    end
    return fc
end

function write_aggregate(name, final_clusters)
    println("writing aggregate files")
    json_string = JSON.json(final_clusters)
    open("aggregate/final_clusters $name.json","w") do file
        write(file, json_string)
    end
end

write_aggregate (generic function with 1 method)

In [36]:
for g in [0.5, 0.75]
    name = "media mo[0.05, 0.5, 0.95] p0.1 e0.2 g$g gm$g mi1000000"
    f = deffuant_bias_media
    final_clusters = return_dictionaries(f, name; nruns=100)
    write_aggregate(name, final_clusters)
    println(read_json_cluster(filename))
end

Dict{String, Any}("32" => Dict{String, Any}("(Float16(0.9473), Float16(0.957))" => 24, "(Float16(0.05008), Float16(0.0601))" => 29, "(Float16(0.4995), Float16(0.5093))" => 47), "29" => Dict{String, Any}("(Float16(0.05038), Float16(0.0604))" => 33, "(Float16(0.949), Float16(0.959))" => 31, "(Float16(0.4998), Float16(0.51))" => 36), "81" => Dict{String, Any}("(Float16(0.05017), Float16(0.06018))" => 7, "(Float16(0.948), Float16(0.958))" => 22, "(Float16(0.4976), Float16(0.5073))" => 71), "54" => Dict{String, Any}("(Float16(0.0501), Float16(0.06012))" => 35, "(Float16(0.5005), Float16(0.5103))" => 36, "(Float16(0.9487), Float16(0.9585))" => 29), "78" => Dict{String, Any}("(Float16(0.949), Float16(0.959))" => 24, "(Float16(0.4995), Float16(0.5093))" => 56, "(Float16(0.05057), Float16(0.06058))" => 20), "1" => Dict{String, Any}("(Float16(0.05048), Float16(0.0605))" => 3, "(Float16(0.9487), Float16(0.9585))" => 24, "(Float16(0.4976), Float16(0.5073))" => 73), "2" => Dict{String, Any}("(Float

(0.948), Float16(0.958))" => 25, "(Float16(0.5), Float16(0.51))" => 56), "64" => Dict{String, Any}("(Float16(0.05002), Float16(0.06003))" => 32, "(Float16(0.9487), Float16(0.9585))" => 44, "(Float16(0.5), Float16(0.51))" => 24), "19" => Dict{String, Any}("(Float16(0.0501), Float16(0.06012))" => 37, "(Float16(0.5005), Float16(0.5103))" => 40, "(Float16(0.9487), Float16(0.9585))" => 23), "91" => Dict{String, Any}("(Float16(0.5005), Float16(0.5103))" => 76, "(Float16(0.95), Float16(0.96))" => 4, "(Float16(0.05045), Float16(0.06046))" => 20), "69" => Dict{String, Any}("(Float16(0.05072), Float16(0.06073))" => 45, "(Float16(0.9478), Float16(0.9575))" => 36, "(Float16(0.5), Float16(0.51))" => 19), "60" => Dict{String, Any}("(Float16(0.95), Float16(0.96))" => 28, "(Float16(0.7334), Float16(0.743))" => 2, "(Float16(0.4976), Float16(0.5073))" => 70), "97" => Dict{String, Any}("(Float16(0.05), Float16(0.06))" => 5, "(Float16(0.9473), Float16(0.957))" => 48, "(Float16(0.499), Float16(0.509))" => 

48), Float16(0.958))" => 25, "(Float16(0.05032), Float16(0.06033))" => 29, "(Float16(0.5), Float16(0.51))" => 46), "59" => Dict{String, Any}("(Float16(0.4985), Float16(0.5083))" => 51, "(Float16(0.05008), Float16(0.0601))" => 14, "(Float16(0.948), Float16(0.958))" => 35), "5" => Dict{String, Any}("(Float16(0.05002), Float16(0.06003))" => 17, "(Float16(0.9487), Float16(0.9585))" => 39, "(Float16(0.4995), Float16(0.5093))" => 44), "31" => Dict{String, Any}("(Float16(0.05), Float16(0.06))" => 16, "(Float16(0.948), Float16(0.958))" => 39, "(Float16(0.4995), Float16(0.5093))" => 45), "89" => Dict{String, Any}("(Float16(0.9478), Float16(0.9575))" => 30, "(Float16(0.05002), Float16(0.06003))" => 26, "(Float16(0.499), Float16(0.509))" => 44), "62" => Dict{String, Any}("(Float16(0.0503), Float16(0.0603))" => 29, "(Float16(0.5005), Float16(0.5103))" => 67, "(Float16(0.95), Float16(0.96))" => 4), "43" => Dict{String, Any}("(Float16(0.7354), Float16(0.745))" => 50, "(Float16(0.0501), Float16(0.060

, "18" => Dict{String, Any}("(Float16(0.05005), Float16(0.06006))" => 29, "(Float16(0.5005), Float16(0.5103))" => 46, "(Float16(0.948), Float16(0.958))" => 25), "30" => Dict{String, Any}("(Float16(0.949), Float16(0.959))" => 16, "(Float16(0.5005), Float16(0.5103))" => 57, "(Float16(0.05014), Float16(0.06015))" => 27), "3" => Dict{String, Any}("(Float16(0.9473), Float16(0.957))" => 40, "(Float16(0.0505), Float16(0.06052))" => 31, "(Float16(0.499), Float16(0.509))" => 29))
Dict{Any, Any}(5 => Dict{Tuple{Float16, Float16}, Int64}((0.4995, 0.5093) => 44, (0.05002, 0.06003) => 17, (0.9487, 0.9585) => 39), 56 => Dict{Tuple{Float16, Float16}, Int64}((0.05014, 0.06015) => 27, (0.948, 0.958) => 8, (0.5005, 0.5103) => 65), 55 => Dict{Tuple{Float16, Float16}, Int64}((0.05, 0.06) => 34, (0.5, 0.51) => 16, (0.9487, 0.9585) => 50), 35 => Dict{Tuple{Float16, Float16}, Int64}((0.05017, 0.06018) => 25, (0.948, 0.958) => 27, (0.499, 0.509) => 48), 60 => Dict{Tuple{Float16, Float16}, Int64}((0.95, 0.96) 

((0.05014, 0.06015) => 42, (0.5, 0.51) => 46, (0.949, 0.959) => 12), 99 => Dict{Tuple{Float16, Float16}, Int64}((0.9473, 0.957) => 36, (0.05035, 0.06036) => 44, (0.5, 0.51) => 20), 89 => Dict{Tuple{Float16, Float16}, Int64}((0.499, 0.509) => 44, (0.9478, 0.9575) => 30, (0.05002, 0.06003) => 26), 14 => Dict{Tuple{Float16, Float16}, Int64}((0.5, 0.51) => 1, (0.74, 0.75) => 51, (0.0501, 0.06012) => 48), 80 => Dict{Tuple{Float16, Float16}, Int64}((0.05032, 0.06033) => 21, (0.499, 0.509) => 48, (0.9487, 0.9585) => 31), 3 => Dict{Tuple{Float16, Float16}, Int64}((0.0505, 0.06052) => 31, (0.9473, 0.957) => 40, (0.499, 0.509) => 29), 96 => Dict{Tuple{Float16, Float16}, Int64}((0.948, 0.958) => 41, (0.4995, 0.5093) => 23, (0.0504, 0.06042) => 36), 51 => Dict{Tuple{Float16, Float16}, Int64}((0.9478, 0.9575) => 23, (0.0501, 0.06012) => 36, (0.5005, 0.5103) => 41), 33 => Dict{Tuple{Float16, Float16}, Int64}((0.948, 0.958) => 33, (0.499, 0.509) => 48, (0.0503, 0.0603) => 19), 40 => Dict{Tuple{Float1

 => 25, "(Float16(0.05032), Float16(0.06033))" => 31, "(Float16(0.5), Float16(0.51))" => 44), "20" => Dict{String, Any}("(Float16(0.05048), Float16(0.0605))" => 33, "(Float16(0.948), Float16(0.958))" => 30, "(Float16(0.5), Float16(0.51))" => 37), "6" => Dict{String, Any}("(Float16(0.4985), Float16(0.5083))" => 68, "(Float16(0.948), Float16(0.958))" => 23, "(Float16(0.05084), Float16(0.06085))" => 9), "84" => Dict{String, Any}("(Float16(0.949), Float16(0.959))" => 25, "(Float16(0.05005), Float16(0.06006))" => 27, "(Float16(0.5005), Float16(0.5103))" => 48), "80" => Dict{String, Any}("(Float16(0.949), Float16(0.959))" => 30, "(Float16(0.0501), Float16(0.06012))" => 43, "(Float16(0.5), Float16(0.51))" => 27), "98" => Dict{String, Any}("(Float16(0.4985), Float16(0.5083))" => 74, "(Float16(0.0508), Float16(0.06082))" => 1, "(Float16(0.948), Float16(0.958))" => 25), "70" => Dict{String, Any}("(Float16(0.05038), Float16(0.0604))" => 19, "(Float16(0.9473), Float16(0.957))" => 34, "(Float16(0.4

Float16(0.509))" => 66), "63" => Dict{String, Any}("(Float16(0.05005), Float16(0.06006))" => 27, "(Float16(0.948), Float16(0.958))" => 44, "(Float16(0.5), Float16(0.51))" => 29), "92" => Dict{String, Any}("(Float16(0.0502), Float16(0.0602))" => 32, "(Float16(0.95), Float16(0.96))" => 4, "(Float16(0.5005), Float16(0.5103))" => 64), "88" => Dict{String, Any}("(Float16(0.4983), Float16(0.5083))" => 36, "(Float16(0.05), Float16(0.06))" => 19, "(Float16(0.948), Float16(0.958))" => 45), "93" => Dict{String, Any}("(Float16(0.4985), Float16(0.5083))" => 59, "(Float16(0.948), Float16(0.958))" => 41), "26" => Dict{String, Any}("(Float16(0.0502), Float16(0.0602))" => 23, "(Float16(0.5005), Float16(0.5103))" => 56, "(Float16(0.948), Float16(0.958))" => 21), "10" => Dict{String, Any}("(Float16(0.0501), Float16(0.06012))" => 22, "(Float16(0.948), Float16(0.958))" => 37, "(Float16(0.4995), Float16(0.5093))" => 41), "77" => Dict{String, Any}("(Float16(0.949), Float16(0.959))" => 11, "(Float16(0.5005),

6(0.959))" => 49, "(Float16(0.0504), Float16(0.06042))" => 27, "(Float16(0.499), Float16(0.509))" => 24), "38" => Dict{String, Any}("(Float16(0.0503), Float16(0.0603))" => 52, "(Float16(0.948), Float16(0.958))" => 21, "(Float16(0.5), Float16(0.51))" => 27), "72" => Dict{String, Any}("(Float16(0.05005), Float16(0.06006))" => 16, "(Float16(0.5005), Float16(0.5103))" => 72, "(Float16(0.948), Float16(0.958))" => 12), "83" => Dict{String, Any}("(Float16(0.949), Float16(0.959))" => 12, "(Float16(0.05014), Float16(0.06015))" => 42, "(Float16(0.5), Float16(0.51))" => 46), "45" => Dict{String, Any}("(Float16(0.5005), Float16(0.5103))" => 77, "(Float16(0.05035), Float16(0.06036))" => 23), "68" => Dict{String, Any}("(Float16(0.05017), Float16(0.06018))" => 37, "(Float16(0.948), Float16(0.958))" => 13, "(Float16(0.5), Float16(0.51))" => 50), "56" => Dict{String, Any}("(Float16(0.5005), Float16(0.5103))" => 65, "(Float16(0.05014), Float16(0.06015))" => 27, "(Float16(0.948), Float16(0.958))" => 8), 

((0.05014, 0.06015) => 36, (0.5, 0.51) => 34, (0.948, 0.958) => 30), 46 => Dict{Tuple{Float16, Float16}, Int64}((0.4985, 0.5083) => 65, (0.05017, 0.06018) => 5, (0.9478, 0.9575) => 30), 57 => Dict{Tuple{Float16, Float16}, Int64}((0.05, 0.06) => 1, (0.5, 0.51) => 19, (0.9487, 0.9585) => 35, (0.2925, 0.3025) => 45), 70 => Dict{Tuple{Float16, Float16}, Int64}((0.05, 0.06) => 29, (0.501, 0.5107) => 70, (0.95, 0.96) => 1), 21 => Dict{Tuple{Float16, Float16}, Int64}((0.499, 0.509) => 24, (0.0504, 0.06042) => 27, (0.949, 0.959) => 49), 88 => Dict{Tuple{Float16, Float16}, Int64}((0.05, 0.06) => 19, (0.4983, 0.5083) => 36, (0.948, 0.958) => 45), 38 => Dict{Tuple{Float16, Float16}, Int64}((0.5, 0.51) => 27, (0.948, 0.958) => 21, (0.0503, 0.0603) => 52), 78 => Dict{Tuple{Float16, Float16}, Int64}((0.05057, 0.06058) => 20, (0.4995, 0.5093) => 56, (0.949, 0.959) => 24), 72 => Dict{Tuple{Float16, Float16}, Int64}((0.05005, 0.06006) => 16, (0.948, 0.958) => 12, (0.5005, 0.5103) => 72), 24 => Dict{Tup

Dict{Any, Any} with 100 entries:
  5  => Dict{Tuple{Float16, Float16}, Int64}((0.4995, 0.5093)=>44, (0.05002, 0.…
  56 => Dict{Tuple{Float16, Float16}, Int64}((0.05014, 0.06015)=>27, (0.948, 0.…
  55 => Dict{Tuple{Float16, Float16}, Int64}((0.05, 0.06)=>34, (0.5, 0.51)=>16,…
  35 => Dict{Tuple{Float16, Float16}, Int64}((0.05017, 0.06018)=>25, (0.948, 0.…
  60 => Dict{Tuple{Float16, Float16}, Int64}((0.95, 0.96)=>28, (0.7334, 0.743)=…
  30 => Dict{Tuple{Float16, Float16}, Int64}((0.05014, 0.06015)=>27, (0.949, 0.…
  32 => Dict{Tuple{Float16, Float16}, Int64}((0.9473, 0.957)=>24, (0.4995, 0.50…
  6  => Dict{Tuple{Float16, Float16}, Int64}((0.05, 0.06)=>13, (0.948, 0.958)=>…
  67 => Dict{Tuple{Float16, Float16}, Int64}((0.498, 0.508)=>23, (0.0504, 0.060…
  45 => Dict{Tuple{Float16, Float16}, Int64}((0.05035, 0.06036)=>23, (0.5005, 0…
  73 => Dict{Tuple{Float16, Float16}, Int64}((0.05035, 0.06036)=>19, (0.4995, 0…
  64 => Dict{Tuple{Float16, Float16}, Int64}((0.5, 0.51)=>24, (0.05002, 0.06

In [37]:
read_json_cluster(filename)

Dict{String, Any}("32" => Dict{String, Any}("(Float16(0.9473), Float16(0.957))" => 24, "(Float16(0.05008), Float16(0.0601))" => 29, "(Float16(0.4995), Float16(0.5093))" => 47), "29" => Dict{String, Any}("(Float16(0.05038), Float16(0.0604))" => 33, "(Float16(0.949), Float16(0.959))" => 31, "(Float16(0.4998), Float16(0.51))" => 36), "81" => Dict{String, Any}("(Float16(0.05017), Float16(0.06018))" => 7, "(Float16(0.948), Float16(0.958))" => 22, "(Float16(0.4976), Float16(0.5073))" => 71), "54" => Dict{String, Any}("(Float16(0.0501), Float16(0.06012))" => 35, "(Float16(0.5005), Float16(0.5103))" => 36, "(Float16(0.9487), Float16(0.9585))" => 29), "78" => Dict{String, Any}("(Float16(0.949), Float16(0.959))" => 24, "(Float16(0.4995), Float16(0.5093))" => 56, "(Float16(0.05057), Float16(0.06058))" => 20), "1" => Dict{String, Any}("(Float16(0.05048), Float16(0.0605))" => 3, "(Float16(0.9487), Float16(0.9585))" => 24, "(Float16(0.4976), Float16(0.5073))" => 73), "2" => Dict{String, Any}("(Float

0.948), Float16(0.958))" => 25, "(Float16(0.5), Float16(0.51))" => 56), "64" => Dict{String, Any}("(Float16(0.05002), Float16(0.06003))" => 32, "(Float16(0.9487), Float16(0.9585))" => 44, "(Float16(0.5), Float16(0.51))" => 24), "19" => Dict{String, Any}("(Float16(0.0501), Float16(0.06012))" => 37, "(Float16(0.5005), Float16(0.5103))" => 40, "(Float16(0.9487), Float16(0.9585))" => 23), "91" => Dict{String, Any}("(Float16(0.5005), Float16(0.5103))" => 76, "(Float16(0.95), Float16(0.96))" => 4, "(Float16(0.05045), Float16(0.06046))" => 20), "69" => Dict{String, Any}("(Float16(0.05072), Float16(0.06073))" => 45, "(Float16(0.9478), Float16(0.9575))" => 36, "(Float16(0.5), Float16(0.51))" => 19), "60" => Dict{String, Any}("(Float16(0.95), Float16(0.96))" => 28, "(Float16(0.7334), Float16(0.743))" => 2, "(Float16(0.4976), Float16(0.5073))" => 70), "97" => Dict{String, Any}("(Float16(0.05), Float16(0.06))" => 5, "(Float16(0.9473), Float16(0.957))" => 48, "(Float16(0.499), Float16(0.509))" => 4

Dict{Any, Any} with 100 entries:
  5  => Dict{Tuple{Float16, Float16}, Int64}((0.4995, 0.5093)=>44, (0.05002, 0.…
  56 => Dict{Tuple{Float16, Float16}, Int64}((0.05014, 0.06015)=>27, (0.948, 0.…
  55 => Dict{Tuple{Float16, Float16}, Int64}((0.05, 0.06)=>34, (0.5, 0.51)=>16,…
  35 => Dict{Tuple{Float16, Float16}, Int64}((0.05017, 0.06018)=>25, (0.948, 0.…
  60 => Dict{Tuple{Float16, Float16}, Int64}((0.95, 0.96)=>28, (0.7334, 0.743)=…
  30 => Dict{Tuple{Float16, Float16}, Int64}((0.05014, 0.06015)=>27, (0.949, 0.…
  32 => Dict{Tuple{Float16, Float16}, Int64}((0.9473, 0.957)=>24, (0.4995, 0.50…
  6  => Dict{Tuple{Float16, Float16}, Int64}((0.05, 0.06)=>13, (0.948, 0.958)=>…
  67 => Dict{Tuple{Float16, Float16}, Int64}((0.498, 0.508)=>23, (0.0504, 0.060…
  45 => Dict{Tuple{Float16, Float16}, Int64}((0.05035, 0.06036)=>23, (0.5005, 0…
  73 => Dict{Tuple{Float16, Float16}, Int64}((0.05035, 0.06036)=>19, (0.4995, 0…
  64 => Dict{Tuple{Float16, Float16}, Int64}((0.5, 0.51)=>24, (0.05002, 0.06